In [ ]:
import os
import numpy as np
from numpy import abs, max 
import scipy
from scipy import interpolate
from matplotlib import pyplot as plt
import sys
import pandas as pd
import random
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import glob

import keras
from keras.layers import Input,LSTM,Dense,Dropout,Activation,noise,normalization,TimeDistributed,Flatten,Masking,Embedding,Conv1D, MaxPooling1D, RepeatVector, Permute, Lambda, Multiply,Reshape
from keras.models import Sequential, Model, load_model
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import EarlyStopping
from IPython.display import SVG
import keras_utils
from keras.utils.np_utils import to_categorical
from sklearn.metrics import mean_absolute_error
from keras import backend as K

from tensorflow import keras

In [ ]:
subject_list = ['A','B','C','D','E','F','G','H','I']
subject_list1 = subject_list[:3]
subject_list2 = subject_list[3:]
action_list = ['pushup','squat','abs']
action_dict = {'pushup':'p','squat':'s','abs':'a'}
target = 'A'
if target in subject_list1:
    subject_list1.remove(target)
if target in subject_list2:
    subject_list2.remove(target)
print(subject_list1)
print(subject_list2)

In [ ]:
use_num = 15
aug_num = 100
aug_num_l = 100

In [ ]:
def make_base_dataset(subject,use_id,aug_num):
    dataset = np.zeros((0,3,35000))
    cnt_pushup = np.zeros(0)
    cnt_squat = np.zeros(0)
    cnt_abs = np.zeros(0)
    
    for i in use_id:
        npz = np.load('../data/raw_data/'+subject+'/'+str(i)+'.npz')
        acc = npz['d']
        c_p = npz['p']
        c_s = npz['s']
        c_a = npz['a']
        
        #zero padding
        data = np.zeros((1,3,35000))
        data[0,:,:acc.shape[1]] = acc
        
        #append
        dataset = np.append(dataset,data,axis=0)
        cnt_pushup = np.append(cnt_pushup,c_p)
        cnt_squat = np.append(cnt_squat,c_s)
        cnt_abs = np.append(cnt_abs,c_a)
        
        #Data Augmentation
        npz_aug = np.load('../data/augmented_data/'+subject+'/'+str(i)+'/'+str(aug_num)+'.npz')
        data_aug = npz_aug['d']
        c_p_aug = npz_aug['p']
        c_s_aug = npz_aug['s']
        c_a_aug = npz_aug['a']
        
        #append
        dataset = np.append(dataset,data_aug,axis=0)
        cnt_pushup = np.append(cnt_pushup,c_p_aug)
        cnt_squat = np.append(cnt_squat,c_s_aug)        
        cnt_abs = np.append(cnt_abs,c_a_aug)
        
    return dataset,cnt_pushup,cnt_squat,cnt_abs

In [ ]:
base_dataset = np.zeros((0,3,35000))
label_p = np.zeros(0)
label_s = np.zeros(0)
label_a = np.zeros(0)

train_idx = {}

for subject in subject_list1:
    use_idx = []
    while len(use_idx) < use_num:
        r = random.randint(0,19)
        if not r in use_idx:
            use_idx.append(r)
    train_idx[subject] = use_idx
    
    print('loading dataset :',subject,)
    dataset,cnt_p,cnt_s,cnt_a = make_base_dataset(subject,use_idx,aug_num)
    
    base_dataset = np.append(base_dataset,dataset,axis=0)
    label_p = np.append(label_p,cnt_p)
    label_s = np.append(label_s,cnt_s)
    label_a = np.append(label_a,cnt_a)
    
for subject in subject_list2:
    use_idx = [i for i in range(5)]
    print('loading dataset :',subject,)
    dataset,cnt_p,cnt_s,cnt_a = make_base_dataset(subject,use_idx,aug_num)
    
    base_dataset = np.append(base_dataset,dataset,axis=0)
    label_p = np.append(label_p,cnt_p)
    label_s = np.append(label_s,cnt_s)
    label_a = np.append(label_a,cnt_a)

In [ ]:
label_dic = {'p':label_p,'s':label_s,'a':label_a}
print(base_dataset.shape)

### Val Data
if target subject is A, B, or C, comment out the code for preparation of the relevant validation data

In [ ]:
dataset2 = np.zeros((0,3,35000))
cnt_pushup2 = np.zeros(0)
cnt_squat2 = np.zeros(0)
cnt_abs2 = np.zeros(0)

#Data of the remaining suject A
# val_idx = list(range(40))
# for i in train_idx['A']:
#     val_idx.remove(i)

# for i in val_idx:   
    
#     npz2_n = np.load('../data/raw_data/A/'+str(i)+'.npz')
#     acc2_n = npz2_n['d']
#     cnt_p2_n = npz2_n['p']
#     cnt_s2_n = npz2_n['s']
#     cnt_a2_n = npz2_n['a']
    
#     Data2_n = np.zeros((1,3,35000))
#     Data2_n[0,:,:acc2_n.shape[1]] = acc2_n
    
#     dataset2 = np.append(dataset2,Data2_n,axis=0)
#     cnt_pushup2 = np.append(cnt_pushup2,cnt_p2_n)
#     cnt_squat2 = np.append(cnt_squat2,cnt_s2_n)
#     cnt_abs2 = np.append(cnt_abs2,cnt_a2_n)

    
#Data of the remaining suject B
val_idx = list(range(20))
for i in train_idx['B']:
    val_idx.remove(i)
for i in val_idx:
    npz2_t = np.load('../data/raw_data/B/'+str(i)+'.npz')
    acc2_t = npz2_t['d']
    cnt_p2_t = npz2_t['p']
    cnt_s2_t = npz2_t['s']
    cnt_a2_t = npz2_t['a']
    
    Data2_t = np.zeros((1,3,35000))
    Data2_t[0,:,:acc2_t.shape[1]] = acc2_t
    
    dataset2 = np.append(dataset2,Data2_t,axis=0)
    cnt_pushup2 = np.append(cnt_pushup2,cnt_p2_t)
    cnt_squat2 = np.append(cnt_squat2,cnt_s2_t)
    cnt_abs2 = np.append(cnt_abs2,cnt_a2_t)


#Data of the remaining suject C
val_idx = list(range(20))
for i in train_idx['C']:
    val_idx.remove(i)
for i in val_idx:
    npz2_t = np.load('../data/raw_data/C/'+str(i)+'.npz')
    acc2_t = npz2_t['d']
    cnt_p2_t = npz2_t['p']
    cnt_s2_t = npz2_t['s']
    cnt_a2_t = npz2_t['a']
    
    Data2_t = np.zeros((1,3,35000))
    Data2_t[0,:,:acc2_t.shape[1]] = acc2_t
    
    dataset2 = np.append(dataset2,Data2_t,axis=0)
    cnt_pushup2 = np.append(cnt_pushup2,cnt_p2_t)
    cnt_squat2 = np.append(cnt_squat2,cnt_s2_t)
    cnt_abs2 = np.append(cnt_abs2,cnt_a2_t)
    
    
print(dataset2.shape)

In [ ]:
label_dict_val = {'p':cnt_pushup2,'s':cnt_squat2,'a':cnt_abs2}
num_data2 = dataset2.shape[0]
for i in range(num_data2):
    for j in range(3):
        dataset2[i][j] = preprocessing.scale(dataset2[i][j])
        
dataset2 = dataset2.transpose(0,2,1)
print(dataset2.shape)

### make LD dataset

In [ ]:
def make_DA_dataset(subject,act,use_id,aug_num,aug_num_l,):
    dataset = np.zeros((0,3,35000))
    cnt = np.zeros(0)
    #double and triple
    for i in use_id:
        for times in ['double','triple']:
            path = '../data/'+times+'/'+subject+'/'+str(i)+'.npz'
            if os.path.exists(path) == True:
                npz = np.load(path)
                acc = npz['d']
                c = npz[act]

                if c <= 20:
                    data = np.zeros((1,3,35000))
                    data[0,:,:acc.shape[1]] = acc

                    dataset = np.append(dataset,data,axis=0)
                    cnt = np.append(cnt,c)

                    npz_aug = np.load('../data/augmented_data/'+subject+'/'+times+'/'+str(i)+'/'+str(aug_num_l)+'.npz')
                    data_aug = npz_aug['d']
                    cnt_aug = npz_aug[act]

                    dataset = np.append(dataset,data_aug,axis=0)
                    cnt = np.append(cnt,cnt_aug)
                
    #plus
    use_id.sort()
    use_id2 = use_id.copy()
    for i in use_id:
        use_id2.remove(i)
        for j in use_id2:
            plus_id = str(i)+'+'+str(j)
            path = '../data/plus2/'+subject+'/'+plus_id+'.npz'
            if os.path.exists(path) == True:
                npz = np.load(path)
                acc = npz['d']
                c = npz[act]
                
                if c <= 20:
                    data = np.zeros((1,3,35000))
                    data[0,:,:acc.shape[1]] = acc
                    
                    dataset = np.append(dataset,data,axis=0)
                    cnt = np.append(cnt,c)
                    
                    npz_aug = np.load('../data/augmented_data/'+subject+'/plus2/'+plus_id+'/'+str(aug_num_l)+'.npz')
                    data_aug = npz_aug['d']
                    cnt_aug = npz_aug[act]
                    
                    dataset = np.append(dataset,data_aug,axis=0)
                    cnt = np.append(cnt,cnt_aug)
                    
    return dataset,cnt

### Model

In [ ]:
hidden_unit = 8
kernel_size = 4

_input = Input(shape=(35000,3))
mask = Masking(mask_value=0.0)(_input)

conv_l = Conv1D(hidden_unit,8,padding='same',activation='relu')(mask)
conv_l = MaxPooling1D()(conv_l)
conv_l = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(conv_l)
conv_l = MaxPooling1D()(conv_l)
conv_l = Conv1D(hidden_unit,2,padding='same',activation='relu')(conv_l)
conv_l = MaxPooling1D()(conv_l)
conv_l = Dropout(0.5)(conv_l)

attention = Conv1D(hidden_unit,8,padding='same',activation='relu')(conv_l)
attention = Dropout(0.5)(attention)
attention = Conv1D(hidden_unit,8,padding='same',activation='relu')(attention)
attention = Activation('sigmoid')(attention)

sent_representation = Multiply()([conv_l, attention])
sent_representation = Dropout(0.5)(sent_representation)

x = Lambda(lambda xin: K.sum(xin,axis=2))(sent_representation)
x = Reshape((-1,1))(x)
x = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(x)
x = MaxPooling1D()(x)
x = Conv1D(hidden_unit,kernel_size,padding='same',activation='relu')(x)
x = MaxPooling1D()(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(hidden_unit,activation='relu')(x)
_output = Dense(1,activation='linear')(x)

### Roop of training on each three acitons

In [ ]:
time1 = time.time()
model_dic = {}
mae_dic = {}

for action in action_list:
    print(action)
    act = action_dict[action]
    
    # train data
    X = base_dataset
    y = label_dic[act]

    for subject in subject_list1:
        use_idx = train_idx[subject]
        print('loading dataset :',subject)
        dataset,cnt = make_DA_dataset(subject,act,use_idx,aug_num,aug_num_l)
        X = np.append(X,dataset,axis=0)
        y = np.append(y,cnt)

    for subject in subject_list2:
        use_idx = [i for i in range(5)]
        print('loading dataset :',subject)
        dataset,cnt = make_DA_dataset(subject,act,use_idx,aug_num,aug_num_l)
        X = np.append(X,dataset,axis=0)
        y = np.append(y,cnt)
        
    p = np.random.permutation(len(y))
    X_train = X[p]
    y_train = y[p]
    
    num_data = X_train.shape[0]
    for i in range(num_data):
        for j in range(3):
            X_train[i][j] = preprocessing.scale(X_train[i][j])

    X_train = X_train.transpose(0,2,1)
    y_train = y_train/20
    
    #validation data
    X_val = dataset2
    y_val = label_dict_val[act]
    y_val = y_val/20
    
    #model compile
    model = Model(inputs=_input, outputs=_output)
    model.compile(loss='mean_absolute_error', optimizer='adam')
    
    nb_epoch = 100
    plot_losses = keras_utils.PlotLosses()
    
    #train model 
    model.fit(X_train,y_train,epochs=nb_epoch,validation_data = (X_val,y_val),callbacks=[plot_losses],verbose=1)
    
    #validation
    predict = model.predict(X_val)
    predict = predict[:,0]
    mae = mean_absolute_error(np.round(predict*20),y_val*20)
    
    model_dic[action] = model
    mae_dic[action] = mae
    
    #save model
    model.save('../data/pre-trained_model/wo_'+target+'_WeakCounter/'+str(aug_num)+action+'.hdf5')
    
time2 = time.time()

In [ ]:
print(time2-time1)

In [ ]:
print(mae_dic)